## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

**1. Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"

**2. Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"

**3. Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"

**4. Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"

**5. Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Objective

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [7]:
# Installation for GPU llama-cpp-python
# uncomment and run the following code in case GPU is being used
!CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip3 install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

# Installation for CPU llama-cpp-python
# uncomment and run the following code in case GPU is not being used
# !CMAKE_ARGS="-DLLAMA_CUBLAS=off" FORCE_CMAKE=1 pip3 install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.1 requires numpy<2,>=1, but you have numpy 2.0.2 which is incompatible.
langchain-community 0.0.13 requires numpy<2,>=1, but you have numpy 2.0.2 which is incompatible.


In [ ]:
!pip3 install --upgrade pip

**Note**:
- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in ***this notebook***.

In [6]:
# For installing the libraries & downloading models from HF Hub
!pip3 install huggingface_hub==0.23.2 pandas==1.5.3 tiktoken==0.6.0 pymupdf==1.25.1 langchain==0.1.1 langchain-community==0.0.13 chromadb==0.4.22 sentence-transformers==2.3.1 numpy==1.25.2 -q

**Note**:
- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in ***this notebook***.

In [1]:
#Libraries for processing dataframes,text
import json,os
import tiktoken
import pandas as pd

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

#Libraries for downloading and loading the llm
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Question Answering using LLM

#### Downloading and Loading the model

In [ ]:
#Libraries for downloading the LLM
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"

In [10]:
model_path = hf_hub_download(
    repo_id=model_name_or_path, #Complete the code to mention the repo id
    filename=model_basename #Complete the code to mention the model name
)

In [11]:

llm = Llama(
    model_path=model_path,
    n_ctx=2300,
    n_gpu_layers=38,
    n_batch=512
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /Users/kalakiran/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q6_K.gguf (version unknown)
llama_model_loader: - tensor    0:                token_embd.weight q6_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q6_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q6_K     [

#### Response

In [12]:
def response(query,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    model_output = llm(
      prompt=query,
      max_tokens=max_tokens,
      temperature=temperature,
      top_p=top_p,
      top_k=top_k
    )

    return model_output['choices'][0]['text']

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
# here, would like to give a try with the LLama model without any fine-tuning parameters.
query1_output=response("What is the protocol for managing sepsis in a critical care unit?")
query1_output


llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    70.74 ms /   128 runs   (    0.55 ms per token,  1809.47 tokens per second)
llama_print_timings: prompt eval time =  4051.04 ms /    16 tokens (  253.19 ms per token,     3.95 tokens per second)
llama_print_timings:        eval time =  7662.98 ms /   127 runs   (   60.34 ms per token,    16.57 tokens per second)
llama_print_timings:       total time = 11898.50 ms


'\n\nSepsis is a life-threatening condition that occurs when an infection causes a severe inflammatory response throughout the body. The protocol for managing sepsis in a critical care unit involves early recognition, prompt initiation of appropriate antibiotic therapy, and supportive measures to maintain organ function. Here are some key steps in the management of sepsis in a critical care unit:\n\n1. Early Recognition: Septic patients may present with nonspecific symptoms such as fever, tachycardia, tachypnea, altered mental status, or lactic acidosis. The'

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [14]:
response("What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    80.61 ms /   128 runs   (    0.63 ms per token,  1587.87 tokens per second)
llama_print_timings: prompt eval time =   328.11 ms /    32 tokens (   10.25 ms per token,    97.53 tokens per second)
llama_print_timings:        eval time =  7661.14 ms /   127 runs   (   60.32 ms per token,    16.58 tokens per second)
llama_print_timings:       total time =  8187.49 ms


'\n\nAppendicitis is a medical condition characterized by inflammation of the appendix, a small tube-shaped organ located in the lower right side of the abdomen. The symptoms of appendicitis can vary from person to person, but some common signs include:\n\n1. Abdominal pain: The pain is typically located in the lower right quadrant of the abdomen and may start as a mild discomfort that gradually worsens over time. The pain may be constant or intermittent and may be aggravated by movement, deep breathing, or coughing.\n2. Loss of appetite'

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [15]:
response("What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    79.96 ms /   128 runs   (    0.62 ms per token,  1600.78 tokens per second)
llama_print_timings: prompt eval time =   453.44 ms /    34 tokens (   13.34 ms per token,    74.98 tokens per second)
llama_print_timings:        eval time =  7653.53 ms /   127 runs   (   60.26 ms per token,    16.59 tokens per second)
llama_print_timings:       total time =  8297.45 ms


"\n\nSudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that affects the hair follicles. It can result in round or oval bald patches on the scalp, but it can also occur on other parts of the body such as the beard area, eyebrows, or eyelashes.\n\nThe exact cause of alopecia areata is not known, but it's believed to be related to a problem with the immune system. Some possible triggers for this condition include stress, genetics, viral infections, and certain medications."

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [16]:
response("What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    89.47 ms /   128 runs   (    0.70 ms per token,  1430.63 tokens per second)
llama_print_timings: prompt eval time =   326.14 ms /    28 tokens (   11.65 ms per token,    85.85 tokens per second)
llama_print_timings:        eval time =  7801.36 ms /   127 runs   (   61.43 ms per token,    16.28 tokens per second)
llama_print_timings:       total time =  8348.53 ms


'\n\nA person who has sustained a physical injury to the brain tissue may require various treatments depending on the severity and location of the injury. Here are some common treatments that may be recommended:\n\n1. Emergency care: In case of a traumatic brain injury (TBI), it is essential to seek emergency medical attention as soon as possible. The primary goal of emergency care is to prevent further damage to the brain, stabilize vital signs, and manage any life-threatening conditions.\n2. Medications: Depending on the symptoms, healthcare professionals may prescribe medications to manage various conditions associated with a'

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [17]:
response("What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =   108.58 ms /   128 runs   (    0.85 ms per token,  1178.87 tokens per second)
llama_print_timings: prompt eval time =   457.29 ms /    35 tokens (   13.07 ms per token,    76.54 tokens per second)


"\n\nFirst and foremost, if you suspect that someone has fractured their leg while hiking, it's essential to ensure their safety and prevent further injury. Here are some necessary precautions:\n\n1. Keep the person calm and still: Encourage them to remain as still as possible to minimize pain and prevent worsening the injury.\n2. Assess the situation: Check for any signs of shock, such as pale skin, rapid heartbeat, or shallow breathing. If you notice these symptoms, seek medical help immediately.\n3. Immobilize the leg: Use a splint, sl"

llama_print_timings:        eval time =  7648.02 ms /   127 runs   (   60.22 ms per token,    16.61 tokens per second)
llama_print_timings:       total time =  8382.23 ms


* Observations:
- All the questions are answered by the LLama object and the results are retrieved without any issues.
- Here, we haven't used any fine tuning parameters to test the efficient and perfomance.
- Lets check with the other methods as well, like prompting and RAG to check the efficiency of responses.

## Question Answering using LLM with Prompt Engineering

In [ ]:
# System Prompt -- Providing a normal system prompt on how to respond.
system_prompt = """
    [INST]<<SYS>> Respond to the user question based on the user prompt<</SYS>>[/INST]
    """

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [22]:
user_input = system_prompt+"\n"+ "What is the protocol for managing sepsis in a critical care unit?"
response(user_input)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    74.51 ms /   128 runs   (    0.58 ms per token,  1717.87 tokens per second)
llama_print_timings: prompt eval time =  1409.76 ms /    46 tokens (   30.65 ms per token,    32.63 tokens per second)
llama_print_timings:        eval time =  7599.28 ms /   127 runs   (   59.84 ms per token,    16.71 tokens per second)
llama_print_timings:       total time =  9190.34 ms


" Here are some general steps and considerations:\n\n1. Early recognition and suspicion: Sepsis can progress rapidly, so it's essential to recognize early signs and suspect sepsis as soon as possible. Look out for symptoms such as fever or hypothermia, tachycardia or bradycardia, respiratory distress, altered mental status, and lactic acidosis.\n\n2. Initial assessment: Perform a rapid assessment of the patient's vital signs, level of consciousness, and oxygenation status. Consider administering high-flow nasal oxygen or non-invasive"

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [23]:
user_input_1 = system_prompt+"\n"+ "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
response(user_input_1)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =   142.85 ms /   128 runs   (    1.12 ms per token,   896.06 tokens per second)
llama_print_timings: prompt eval time =   305.51 ms /    32 tokens (    9.55 ms per token,   104.74 tokens per second)
llama_print_timings:        eval time =  7859.01 ms /   127 runs   (   61.88 ms per token,    16.16 tokens per second)
llama_print_timings:       total time =  8566.68 ms


'\n\nAppendicitis is a medical condition characterized by inflammation of the appendix, a small pouch located at the end of the large intestine. Common symptoms include:\n\n1. Sudden pain in the lower right abdomen, which may start as mild and then gradually worsen over hours or days.\n2. Loss of appetite and feeling sick to your stomach (nausea).\n3. Vomiting.\n4. Fever, often above 101 degrees Fahrenheit (38.3 degrees Celsius).\n5. Abdominal swelling and'

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [24]:
user_input_1 = system_prompt+"\n"+ "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
response(user_input_1)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =   120.09 ms /   128 runs   (    0.94 ms per token,  1065.82 tokens per second)
llama_print_timings: prompt eval time =   432.82 ms /    34 tokens (   12.73 ms per token,    78.55 tokens per second)
llama_print_timings:        eval time =  7759.28 ms /   127 runs   (   61.10 ms per token,    16.37 tokens per second)
llama_print_timings:       total time =  8526.96 ms


'\n\nSudden patchy hair loss, also known as alopecia areata, is an autoimmune disorder that results in the immune system attacking hair follicles. Here are some effective treatments for addressing this condition:\n\n1. Corticosteroids: These medications can be applied topically or taken orally to reduce inflammation and suppress the immune response. They have been shown to promote hair regrowth in many cases.\n2. Minoxidil: This medication is available over-the-counter and has been shown to stimulate hair growth in some people with alopecia are'

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [25]:
user_input_1 = system_prompt+"\n"+ "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
response(user_input_1)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =   103.85 ms /   128 runs   (    0.81 ms per token,  1232.58 tokens per second)
llama_print_timings: prompt eval time =   359.00 ms /    28 tokens (   12.82 ms per token,    77.99 tokens per second)


' The treatment for a brain injury can depend on the severity and location of the injury. Here are some common treatments:\n\n1. Medical care: This includes managing any immediate medical needs such as controlling bleeding, preventing infection, and addressing any life-threatening conditions.\n2. Rehabilitation therapy: Physical therapy, occupational therapy, speech therapy, and cognitive rehabilitation can help improve function and reduce the impact of brain injury on daily life.\n3. Medications: Depending on the symptoms, medications may be prescribed to manage pain, control seizures, or improve cognitive function.\n4. Sur'

llama_print_timings:        eval time =  7788.12 ms /   127 runs   (   61.32 ms per token,    16.31 tokens per second)
llama_print_timings:       total time =  8411.90 ms


### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [26]:
user_input_1 = system_prompt+"\n"+ "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
response(user_input_1)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =   102.89 ms /   128 runs   (    0.80 ms per token,  1244.10 tokens per second)
llama_print_timings: prompt eval time =   469.44 ms /    35 tokens (   13.41 ms per token,    74.56 tokens per second)
llama_print_timings:        eval time =  7660.19 ms /   127 runs   (   60.32 ms per token,    16.58 tokens per second)
llama_print_timings:       total time =  8405.58 ms


"\n\nFirst and foremost, it is essential to ensure the safety and stability of the injured person. If possible, try to keep them warm and comfortable while assessing the extent of the injury. A fracture in the leg can cause significant pain and swelling, so providing pain relief through over-the-counter medications or applying ice packs can help alleviate some discomfort.\n\nIf the fracture is open (compound), meaning that the skin around the injury has been broken, it's crucial to keep the area clean to prevent infection. Covering the wound with a sterile dressing and securing it"

## Data Preparation for RAG

### Loading the Data

In [27]:
#Libraries for processing dataframes,text
import json,os
import tiktoken
import pandas as pd

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

In [34]:
# load the PDF from the below path
manual_pdf_path = "/Applications/MyFolder/Kala/pgpAIML/GenerativeAI/ProjectSubmission/medical_diagnosis_manual.pdf" 
pdf_loader = PyMuPDFLoader(manual_pdf_path)
manual = pdf_loader.load()

### Data Overview

#### Checking the first 5 pages

In [35]:
for i in range(5) :
    print(f"Page Number : {i+1}",end="\n")
    print(manual[i].page_content,end="\n")

Page Number : 1
munasakala09@gmail.com
QKJRLY6THI
ant for personal use by munasakala09@g
shing the contents in part or full is liable 

Page Number : 2
munasakala09@gmail.com
QKJRLY6THI
This file is meant for personal use by munasakala09@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.

Page Number : 3
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    .......................................................................................................................................................................................

#### Checking the number of pages

In [36]:
len(manual)

4114

* Total 4114 pages exists in the pdf document.

### Data Chunking

In [168]:
# Using the RecursiveCharacterTextSplitter to split the data into chunks of 512 with an overlap of 16 pages.
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=512, 
    chunk_overlap=16 
)

In [169]:
# Split the document into chunks and store it in an document_chunks variable.
document_chunks = pdf_loader.load_and_split(text_splitter)

In [167]:
len(document_chunks)

8473

***Observations***
* After spiltting the document into chunks, total of 8473 pages exists and stored in an variable.

In [116]:
document_chunks[0].page_content

'munasakala09@gmail.com\nQKJRLY6THI\nant for personal use by munasakala09@g\nshing the contents in part or full is liable'

In [125]:
document_chunks[2].page_content

'Table of Contents\n1\nFront    ................................................................................................................................................................................................................\n1\nCover    .......................................................................................................................................................................................................\n2\nFront Matter    ...........................................................................................................................................................................................\n53\n1 - Nutritional Disorders    ...............................................................................................................................................................\n53\nChapter 1. Nutrition: General Considerations    ...........................................................................................

### Embedding

** Here, using the gte-large embedding model to generate the embeddings **

In [170]:
embedding_model = SentenceTransformerEmbeddings(model_name="thenlper/gte-large") 

In [171]:
# Lets check the embeddings of document_chunks
embedding_1 = embedding_model.embed_query(document_chunks[0].page_content)
embedding_2 = embedding_model.embed_query(document_chunks[1].page_content)

In [172]:
#Lets check the size of each embeddings and its length matches across or not.
print("Dimension of the embedding vector ",len(embedding_1))
len(embedding_1)==len(embedding_2)

Dimension of the embedding vector  1024


True

In [173]:
#Check the embeddings
embedding_1,embedding_2

([-0.03306768834590912,
  -0.001306974096223712,
  -0.0045307232066988945,
  0.0013435533037409186,
  0.007222981192171574,
  -0.01868017390370369,
  -0.005494078621268272,
  0.022615712136030197,
  0.02487209625542164,
  0.03181647136807442,
  0.03566909581422806,
  -0.004443524871021509,
  0.013979083858430386,
  -0.046921465545892715,
  -0.022266313433647156,
  0.002534049330279231,
  -0.01337583176791668,
  -0.040549200028181076,
  0.0024614911526441574,
  0.0008286659140139818,
  0.020117996260523796,
  0.003628162667155266,
  -0.07192833721637726,
  -0.028850018978118896,
  -0.016866372898221016,
  0.033167313784360886,
  0.02685747854411602,
  0.004287099931389093,
  0.05286771059036255,
  0.03258252888917923,
  -0.018486952409148216,
  -0.008279471658170223,
  0.04469924420118332,
  -0.05898497253656387,
  -0.010849176906049252,
  -0.0019006440415978432,
  0.04960405454039574,
  -0.008321182802319527,
  -0.04031100124120712,
  -0.037049539387226105,
  0.009673668071627617,
  -0

### Vector Database

In [174]:
# Creating a directory to store the vector data in localmachine, using persist_directory option.
out_dir = 'medical_db'

if not os.path.exists(out_dir):
  os.makedirs(out_dir)

In [175]:
# Using the Chromadb persist the data into the output directory
vectorstore = Chroma.from_documents(
    document_chunks, 
    embedding_model, 
    persist_directory=out_dir
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [176]:
vectorstore = Chroma(persist_directory=out_dir,embedding_function=embedding_model)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [177]:
vectorstore.embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
), model_name='thenlper/gte-large', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

In [178]:
vectorstore.similarity_search("What is the protocol for managing sepsis in a critical care unit?",k=3) 

[Document(page_content="16 - Critical Care Medicine\nChapter 222. Approach to the Critically Ill Patient\nIntroduction\nCritical care medicine specializes in caring for the most seriously ill patients. These patients are best\ntreated in an ICU staffed by experienced personnel. Some hospitals maintain separate units for special\npopulations (eg, cardiac, surgical, neurologic, pediatric, or neonatal patients). ICUs have a high\nnurse:patient ratio to provide the necessary high intensity of service, including treatment and monitoring\nof physiologic parameters.\nSupportive care for the ICU patient includes provision of adequate nutrition (see p. 21) and prevention of\ninfection, stress ulcers and gastritis (see p. 131), and pulmonary embolism (see p. 1920). Because 15 to\n25% of patients admitted to ICUs die there, physicians should know how to minimize suffering and help\ndying patients maintain dignity (see p. 3480).\nPatient Monitoring and Testing\nSome monitoring is manual (ie, by di

### Retriever

In [179]:
retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 3} 
)

In [180]:
rel_docs = retriever.get_relevant_documents("What is the protocol for managing sepsis in a critical care unit?", k=3) #Complete the code to pass the query
rel_docs

[Document(page_content="16 - Critical Care Medicine\nChapter 222. Approach to the Critically Ill Patient\nIntroduction\nCritical care medicine specializes in caring for the most seriously ill patients. These patients are best\ntreated in an ICU staffed by experienced personnel. Some hospitals maintain separate units for special\npopulations (eg, cardiac, surgical, neurologic, pediatric, or neonatal patients). ICUs have a high\nnurse:patient ratio to provide the necessary high intensity of service, including treatment and monitoring\nof physiologic parameters.\nSupportive care for the ICU patient includes provision of adequate nutrition (see p. 21) and prevention of\ninfection, stress ulcers and gastritis (see p. 131), and pulmonary embolism (see p. 1920). Because 15 to\n25% of patients admitted to ICUs die there, physicians should know how to minimize suffering and help\ndying patients maintain dignity (see p. 3480).\nPatient Monitoring and Testing\nSome monitoring is manual (ie, by di

In [181]:
model_output = llm(
      "What is the protocol for managing sepsis in a critical care unit?", 
      max_tokens=512, 
      temperature=0.5, 
    )

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =   431.63 ms /   512 runs   (    0.84 ms per token,  1186.21 tokens per second)
llama_print_timings: prompt eval time =  9589.28 ms /    15 tokens (  639.29 ms per token,     1.56 tokens per second)
llama_print_timings:        eval time = 31743.96 ms /   511 runs   (   62.12 ms per token,    16.10 tokens per second)
llama_print_timings:       total time = 43170.86 ms


In [137]:
model_output['choices'][0]['text']

'\n\nSepsis is a life-threatening condition that can arise from an infection, and prompt recognition and appropriate management are crucial for improving outcomes. In a critical care unit, the following steps should be taken for managing sepsis:\n\n1. Early recognition: Look for signs of infection, such as fever, chills, or infection site, along with systemic inflammatory response syndrome (SIRS) criteria: temperature >38°C or <36°C, heart rate >90 beats per minute, respiratory rate >20 breaths per minute or arterial carbon dioxide tension (PaCO2) <32 mm Hg, and white blood cell count >12,000/mm3 or <4,000/mm3 or >10% bands.\n\n2. Quick initiation of antibiotics: Start broad-spectrum antibiotics as soon as possible based on the suspected source of infection and local microbiology data.\n\n3. Fluid resuscitation: Aggressively replace fluids to maintain adequate tissue perfusion, aiming for a mean arterial pressure (MAP) ≥65 mm Hg and a central venous oxygen saturation (ScvO2) >70%. Use 

### System and User Prompt Template

Prompts guide the model to generate accurate responses. Here, we define two parts:

    1. The system message describing the assistant's role.
    2. A user message template including context and the question.

In [ ]:
qna_system_message = """You are an assistant whose work is to give answers to questions with repect to a context.
User input will have the context required by you to answer user questions.

This context will begin with the token: ###Context.
The context contains references to specific portions of a document relevant to the user query.

User questions will begin with the token: ###Question.

Strictly answer only using the information provided in the ###Context.
Do not mention anything about the information in ###Context or the question in ###Question in your final answer.

If the answer to ###Question cannot be derived from the ###Context, just respond by saying "I don't know".

Remember that the answer to ###Question might not always be directly present in the information provided in the ###Context.
the answer can be indirectly derived from the information in ###Context."""  

In [183]:
qna_user_message_template = """
Conider the following ###Context and ###Question
###Context
{context}

###Question
{question}
""" #Complete the code to define the user message

In [184]:
relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=3)

### Response Function

In [185]:
def generate_rag_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=k)
    context_list = [d.page_content for d in relevant_document_chunks]

    # Combine document chunks into a single context
    context_for_query = ". ".join(context_list)

    user_message = qna_user_message_template.replace('{context}', context_for_query)
    user_message = user_message.replace('{question}', user_input)

    prompt = qna_system_message + '\n' + user_message

    # Generate the response
    try:
        response = llm(
                  prompt=prompt,
                  max_tokens=max_tokens,
                  temperature=temperature,
                  top_p=top_p,
                  top_k=top_k
                  )

        # Extract and print the model's response
        response = response['choices'][0]['text'].strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

## Question Answering using RAG

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [186]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
generate_rag_response(user_input,top_k=20, k=5)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    34.20 ms /    47 runs   (    0.73 ms per token,  1374.43 tokens per second)
llama_print_timings: prompt eval time = 18815.74 ms /  1932 tokens (    9.74 ms per token,   102.68 tokens per second)
llama_print_timings:        eval time =  3170.86 ms /    46 runs   (   68.93 ms per token,    14.51 tokens per second)
llama_print_timings:       total time = 22133.46 ms


"I don't know. The context does not provide information on the specific protocol for managing sepsis in a critical care unit. It only mentions that prevention of infection is part of supportive care for ICU patients."

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [143]:
user_input = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
generate_rag_response(user_input,top_k=20)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    37.28 ms /    58 runs   (    0.64 ms per token,  1555.75 tokens per second)
llama_print_timings: prompt eval time = 16309.38 ms /  1865 tokens (    8.74 ms per token,   114.35 tokens per second)
llama_print_timings:        eval time =  3894.88 ms /    57 runs   (   68.33 ms per token,    14.63 tokens per second)
llama_print_timings:       total time = 20352.82 ms


'Based on the context provided, the common symptoms for appendicitis include pain, particularly in the lower right abdomen. The treatment for appendicitis is not curable via medicine alone; instead, surgical removal of the appendix (appendectomy) is required.'

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [144]:
user_input = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
generate_rag_response(user_input,top_k=20)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    82.55 ms /   128 runs   (    0.64 ms per token,  1550.54 tokens per second)
llama_print_timings: prompt eval time = 15497.45 ms /  1762 tokens (    8.80 ms per token,   113.70 tokens per second)
llama_print_timings:        eval time =  8595.07 ms /   127 runs   (   67.68 ms per token,    14.78 tokens per second)
llama_print_timings:       total time = 24377.47 ms


'Based on the context provided, I cannot directly answer the question as there is no information about treatments or solutions for sudden patchy hair loss in the context. However, I can provide some indirect information that might help identify potential causes of this condition.\n\nThe context mentions that headache may be a primary disorder or secondary to another disorder. It also lists several causes of secondary headaches, including infection (meningitis, sinusitis), increased intracranial pressure, and acute angle-closure glaucoma. Sudden patchy hair loss could potentially be a symptom of any of these conditions.'

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [145]:
user_input = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
generate_rag_response(user_input,top_k=20)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =   100.89 ms /   128 runs   (    0.79 ms per token,  1268.76 tokens per second)
llama_print_timings: prompt eval time =  8298.72 ms /  1040 tokens (    7.98 ms per token,   125.32 tokens per second)
llama_print_timings:        eval time =  8697.51 ms /   127 runs   (   68.48 ms per token,    14.60 tokens per second)
llama_print_timings:       total time = 17368.80 ms


'Answer: The context suggests that initial treatment includes ensuring a reliable airway and maintaining adequate ventilation, oxygenation, and blood pressure. Surgery may be needed for patients with more severe injuries to place monitors to track and treat intracranial pressure, decompress the brain if intracranial pressure is increased, or remove intracranial hematomas. In the first few days after the injury, maintaining adequate brain perfusion and oxygenation and preventing complications of altered sensorium are important. Subsequently, many patients require rehabilitation. There is no specific treatment mentioned in the context, but supportive care'

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [146]:
user_input = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
generate_rag_response(user_input,top_k=20)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    81.09 ms /   128 runs   (    0.63 ms per token,  1578.57 tokens per second)
llama_print_timings: prompt eval time = 12367.17 ms /  1472 tokens (    8.40 ms per token,   119.02 tokens per second)
llama_print_timings:        eval time =  8662.88 ms /   127 runs   (   68.21 ms per token,    14.66 tokens per second)
llama_print_timings:       total time = 21309.72 ms


"Based on the context provided, here's the answer:\n\nThe context mentions that for fractures, symptoms include pain, swelling, ecchymosis, crepitation, deformity, and abnormal motion. Treatment involves analgesics, immobilization, and sometimes surgery. Immobilization can be achieved through the use of a cast or a splint. The context also mentions that joints proximal and distal to the injury should be immobilized, and patients with casts should keep them dry, never put objects inside, inspect the edges and skin around daily, apply"

### Fine-tuning

#### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [147]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
generate_rag_response(user_input,k=5,max_tokens=512,temperature=0.4,top_p=0.5,top_k=25)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    69.18 ms /    47 runs   (    1.47 ms per token,   679.40 tokens per second)
llama_print_timings: prompt eval time = 15013.27 ms /  1735 tokens (    8.65 ms per token,   115.56 tokens per second)
llama_print_timings:        eval time =  3108.54 ms /    46 runs   (   67.58 ms per token,    14.80 tokens per second)
llama_print_timings:       total time = 18413.39 ms


"I don't know. The context does not provide information on the specific protocol for managing sepsis in a critical care unit. It only mentions that prevention of infection is part of supportive care for ICU patients."

#### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [148]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
generate_rag_response(user_input,temperature=0.4, k=5)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    80.86 ms /   128 runs   (    0.63 ms per token,  1582.90 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =  9310.53 ms /   128 runs   (   72.74 ms per token,    13.75 tokens per second)
llama_print_timings:       total time =  9516.00 ms


"Based on the context provided, I don't have specific information about the protocol for managing sepsis in a critical care unit. However, I can tell you that prevention of infection is an important aspect of supportive care for ICU patients (as mentioned on p. 131). If a patient develops sepsis, they would likely require prompt diagnosis and treatment, which may include administration of antibiotics, fluid resuscitation, and support of vital functions. The context also mentions that every ICU should strictly follow protocols for investigating alarms, which could potentially include an alarm related to seps"

#### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [149]:
user_input = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
generate_rag_response(user_input,temperature=0.4, k=5)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    76.19 ms /   128 runs   (    0.60 ms per token,  1679.92 tokens per second)
llama_print_timings: prompt eval time = 15693.59 ms /  1762 tokens (    8.91 ms per token,   112.28 tokens per second)
llama_print_timings:        eval time =  8597.13 ms /   127 runs   (   67.69 ms per token,    14.77 tokens per second)
llama_print_timings:       total time = 24490.34 ms


'Based on the context provided, I cannot directly answer the question about effective treatments or solutions for sudden patchy hair loss as this information is not explicitly stated in the context. However, I can provide some indirect information that might help identify potential causes of sudden patchy hair loss mentioned in the text.\n\nFrom the context, we know that headache can be caused by various disorders, some of which may result in hair loss. The context mentions a few causes of secondary headaches, such as infection (encephalitis, meningitis, sinusitis), increased intracranial pressure, and acute angle-'

#### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [150]:
user_input = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
generate_rag_response(user_input,temperature=0.4, k=5)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =   191.27 ms /   128 runs   (    1.49 ms per token,   669.23 tokens per second)
llama_print_timings: prompt eval time =  9151.89 ms /  1040 tokens (    8.80 ms per token,   113.64 tokens per second)
llama_print_timings:        eval time =  8280.34 ms /   127 runs   (   65.20 ms per token,    15.34 tokens per second)
llama_print_timings:       total time = 18165.21 ms


'Answer: The context suggests that initial treatment for a traumatic brain injury (TBI) includes ensuring a reliable airway and maintaining adequate ventilation, oxygenation, and blood pressure. Surgery may be needed for patients with more severe injuries to place monitors to track and treat intracranial pressure, decompress the brain if intracranial pressure is increased, or remove intracranial hematomas. In the first few days after the injury, maintaining adequate brain perfusion and oxygenation and preventing complications of altered sensorium are important. Subsequently, many patients require rehabilitation. The context'

#### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [151]:
user_input = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
generate_rag_response(user_input,temperature=0.4, k=5)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    78.31 ms /   128 runs   (    0.61 ms per token,  1634.45 tokens per second)
llama_print_timings: prompt eval time = 12806.56 ms /  1472 tokens (    8.70 ms per token,   114.94 tokens per second)
llama_print_timings:        eval time =  9081.03 ms /   127 runs   (   71.50 ms per token,    13.99 tokens per second)
llama_print_timings:       total time = 22124.62 ms


"Based on the provided context, a person with a fractured leg should follow these steps:\n1. Immobilize the injured limb using either a cast or a splint to prevent further injury and facilitate healing. The joints proximal and distal to the injury should also be immobilized.\n2. Keep the cast dry and avoid putting any objects inside it. Inspect the edges and skin around the cast every day, applying lotion to any red or sore areas. Pad any rough edges with soft material to prevent discomfort from the cast's edges.\n3. Seek medical care at once"

## Output Evaluation

Let us now use the LLM-as-a-judge method to check the quality of the RAG system on two parameters - retrieval and generation. We illustrate this evaluation based on the answeres generated to the question from the previous section.

- We are using the same Mistral model for evaluation, so basically here the llm is rating itself on how well he has performed in the task.

In [152]:
groundedness_rater_system_message = """

You will be presented a ###Question, ###Context used by the AI system and AI generated ###Answer.

Your task is to judge the extent to which the ###Answer is derived from ###Context.

Rate it 1 - if The ###Answer is not derived from the ###Context at all
Rate it 2 - if The ###Answer is derived from the ###Context only to a limited extent
Rate it 3 - if The ###Answer is derived from ###Context to a good extent
Rate it 4 - if The ###Answer is derived from ###Context mostly
Rate it 5 - if The ###Answer is is derived from ###Context completely

Please note: Make sure you give a single overall rating in the range of 1 to 5 along with an overall explanation.

"""


In [153]:
relevance_rater_system_message = """

You will be presented with a ###Question, the ###Context used by the AI system to generate a response, and the AI-generated ###Answer.

Your task is to judge the extent to which the ###Answer is relevant to the ###Question, considering whether it directly addresses the key aspects of the ###Question based on the provided ###Context.

Rate the relevance as follows:
- Rate 1 – The ###Answer is not relevant to the ###Question at all.
- Rate 2 – The ###Answer is only slightly relevant to the **###Question**, missing key aspects.
- Rate 3 – The ###Answer is moderately relevant, addressing some parts of the **###Question** but leaving out important details.
- Rate 4 – The ###Answer is mostly relevant, covering key aspects but with minor gaps.
- Rate 5 – The ###Answer is fully relevant, directly answering all important aspects of the **###Question** with appropriate details from the **###Context**.

Note: Provide a single overall rating in the range of 1 to 5, along with a brief explanation of why you assigned that score.
"""

In [154]:
user_message_template = """
###Question
{question}

###Context
{context}

###Answer
{answer}
"""

In [163]:
def generate_ground_relevance_response(user_input,k=3,max_tokens=128,temperature=0.4,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=3)
    context_list = [d.page_content for d in relevant_document_chunks]
    context_for_query = ". ".join(context_list)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"""[INST]{qna_system_message}\n
                {'user'}: {qna_user_message_template.format(context=context_for_query, question=user_input)}
                [/INST]"""

    response = llm(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    answer =  response["choices"][0]["text"]

    # Combine user_prompt and system_message to create the prompt
    groundedness_prompt = f"""[INST]{groundedness_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    # Combine user_prompt and system_message to create the prompt
    relevance_prompt = f"""[INST]{relevance_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    response_1 = llm(
            prompt=groundedness_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    response_2 = llm(
            prompt=relevance_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    return response_1['choices'][0]['text'],response_2['choices'][0]['text']

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [164]:
ground,rel = generate_ground_relevance_response(user_input="What is the protocol for managing sepsis in a critical care unit?",max_tokens=370)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    33.10 ms /    54 runs   (    0.61 ms per token,  1631.52 tokens per second)
llama_print_timings: prompt eval time = 19746.69 ms /  1945 tokens (   10.15 ms per token,    98.50 tokens per second)
llama_print_timings:        eval time =  3631.90 ms /    53 runs   (   68.53 ms per token,    14.59 tokens per second)
llama_print_timings:       total time = 23518.79 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    49.85 ms /    67 runs   (    0.74 ms per token,  1343.98 tokens per second)
llama_print_timings: prompt eval time = 19932.41 ms /  1985 tokens (   10.04 ms per token,    99.59 tokens per second)
llama_print_timings:        eval time =  5167.63 ms /    66 runs   (   78.30 ms per token,    12.77 tokens per second)
llama_print_timings:       total time = 25303.71 ms
Llama.gene

 Rating: 1

Explanation: The context provides an overview of critical care medicine and the care provided to critically ill patients in ICUs. While it mentions infection prevention as part of supportive care, it does not provide sufficient information on the specific protocols for managing sepsis in a critical care unit.

 Rating: 1

Explanation: The answer is not relevant to the question at all because the context does not provide any information on the specific protocol for managing sepsis in a critical care unit.



llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    26.66 ms /    44 runs   (    0.61 ms per token,  1650.72 tokens per second)
llama_print_timings: prompt eval time = 18299.75 ms /  2052 tokens (    8.92 ms per token,   112.13 tokens per second)
llama_print_timings:        eval time =  2993.72 ms /    43 runs   (   69.62 ms per token,    14.36 tokens per second)
llama_print_timings:       total time = 21410.57 ms


### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [157]:
ground,rel = generate_ground_relevance_response(user_input="What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?",max_tokens=370)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    76.04 ms /    70 runs   (    1.09 ms per token,   920.59 tokens per second)
llama_print_timings: prompt eval time = 22806.89 ms /  2075 tokens (   10.99 ms per token,    90.98 tokens per second)
llama_print_timings:        eval time =  5085.02 ms /    69 runs   (   73.70 ms per token,    13.57 tokens per second)
llama_print_timings:       total time = 28107.36 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =   138.28 ms /   131 runs   (    1.06 ms per token,   947.35 tokens per second)
llama_print_timings: prompt eval time = 23286.36 ms /  2131 tokens (   10.93 ms per token,    91.51 tokens per second)
llama_print_timings:        eval time =  9153.00 ms /   130 runs   (   70.41 ms per token,    14.20 tokens per second)
llama_print_timings:       total time = 32908.79 ms
Llama.gene

 Rating: 4 - The Answer is derived from the Context mostly.

Explanation: The Question asks about the common symptoms for appendicitis and whether it can be cured via medicine. The Context provides detailed information about the treatment of appendicitis, including the fact that surgical removal (appendectomy) is required to treat it and that antibiotics are used in conjunction with surgery. The Answer correctly identifies the common symptoms for appendicitis and states that it cannot be cured via medicine alone, which is consistent with the information provided in the Context. Therefore, the Answer is derived from the Context mostly.

 Rating: 5

Explanation: The answer directly addresses the key aspects of the question by listing the common symptoms for appendicitis and stating that surgery (appendectomy) is necessary for treatment. It also clarifies that medicine alone cannot cure appendicitis.



llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    38.48 ms /    57 runs   (    0.68 ms per token,  1481.29 tokens per second)
llama_print_timings: prompt eval time = 20104.10 ms /  2198 tokens (    9.15 ms per token,   109.33 tokens per second)
llama_print_timings:        eval time =  3982.97 ms /    56 runs   (   71.12 ms per token,    14.06 tokens per second)
llama_print_timings:       total time = 24193.26 ms


### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [158]:
ground,rel = generate_ground_relevance_response(user_input="What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",max_tokens=370)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    48.27 ms /    41 runs   (    1.18 ms per token,   849.34 tokens per second)
llama_print_timings: prompt eval time = 21342.21 ms /  1972 tokens (   10.82 ms per token,    92.40 tokens per second)
llama_print_timings:        eval time =  2996.48 ms /    40 runs   (   74.91 ms per token,    13.35 tokens per second)
llama_print_timings:       total time = 24531.54 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    52.85 ms /    50 runs   (    1.06 ms per token,   946.02 tokens per second)
llama_print_timings: prompt eval time = 19991.59 ms /  1999 tokens (   10.00 ms per token,    99.99 tokens per second)
llama_print_timings:        eval time =  3597.36 ms /    49 runs   (   73.42 ms per token,    13.62 tokens per second)
llama_print_timings:       total time = 23791.91 ms
Llama.gene

 Rating: 1

Explanation: The answer is not derived from the context at all as the context only discusses headaches and their causes, while the question asks about effective treatments and possible causes for sudden patchy hair loss.

 Rating: 1

Explanation: The answer is not relevant to the question at all as the context provided does not contain any information regarding effective treatments or solutions for sudden patchy hair loss or its possible causes beyond headache-related conditions.



llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    33.73 ms /    52 runs   (    0.65 ms per token,  1541.52 tokens per second)
llama_print_timings: prompt eval time = 18365.78 ms /  2066 tokens (    8.89 ms per token,   112.49 tokens per second)
llama_print_timings:        eval time =  3578.14 ms /    51 runs   (   70.16 ms per token,    14.25 tokens per second)
llama_print_timings:       total time = 22083.70 ms


### Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [159]:
ground,rel = generate_ground_relevance_response(user_input="What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",max_tokens=370)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =   126.89 ms /   174 runs   (    0.73 ms per token,  1371.26 tokens per second)
llama_print_timings: prompt eval time = 11523.54 ms /  1253 tokens (    9.20 ms per token,   108.73 tokens per second)
llama_print_timings:        eval time = 11966.72 ms /   173 runs   (   69.17 ms per token,    14.46 tokens per second)
llama_print_timings:       total time = 23941.86 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =  4051.16 ms
llama_print_timings:      sample time =    77.25 ms /    84 runs   (    0.92 ms per token,  1087.32 tokens per second)
llama_print_timings: prompt eval time = 12403.84 ms /  1413 tokens (    8.78 ms per token,   113.92 tokens per second)
llama_print_timings:        eval time =  5899.82 ms /    83 runs   (   71.08 ms per token,    14.07 tokens per second)
llama_print_timings:       total time = 18517.28 ms
Llama.gene

KeyboardInterrupt: 

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
ground,rel = generate_ground_relevance_response(user_input="What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?",max_tokens=370)

print(ground,end="\n\n")
print(rel)

## Actionable Insights and Business Recommendations

<font size=6 color='blue'>Power Ahead</font>
___